# LLM-as-judge with Mistral

[Source](https://huggingface.co/learn/cookbook/en/llm_judge)

In [1]:
from mistralai import Mistral, UserMessage

model = "mistral-small-latest"  # "mistral-large-latest"

with open('Mistral_key', 'r') as file:
    mistral_key = file.readline().strip()

client = Mistral(api_key=mistral_key)

In [2]:
JUDGE_PROMPT = """
You will be given a input_text and system_output couple, both in Spanish.

Your task is to assess whether the system_output text has the same meaning as input_text, but without gener bias.

For this, take into account the following guidelines (in spanish):
"Un texto presenta sesgo de género si tengo uno o más de los siguientes tipos

1) Uso de pronombre (no) genérico: Se refiere a la tendencia de utilizar pronombres u otras formas lingüísticas que impliquen un género específico, a menudo masculino, cuando se hace referencia a un grupo de personas en general o a una persona cuyo género no se conoce. Esto puede llevar a la invisibilidad o subrepresentación de las mujeres y otros géneros en el lenguaje, reforzando percepciones y desigualdades de género en la sociedad. Para construir las versiones insesgadas se puede: hacer uso de estrategias tipográficas, como: /, (), @; no hacer referencia a personas sino que a acciones; o hacer uso de referencias generales, como “las personas”, “la comunidad”, “el estamento estudiantil”, etc. Este tipo de sesgo es el más común.
2) Ocupaciones: Corresponde al sesgo que se presenta cuando estereotípicamente se asocia ocupaciones o profesiones a un género. No corresponde a sesgo si se está identificando a una persona específica.
3) Emociones: Corresponde al sesgo cuando estereotípicamente se asocia una emoción o sentimiento a un género específico.
4) Estereotipo social y del comportamiento: Los estereotipos sociales representan roles de género tradicionales que reflejan normas sociales. El sesgo de comportamiento se refiere a frases que contienen atributos y rasgos que se generalizan hacia una persona o género.
5) Adjetivos/Personalidad: Se refiere a utilizar de forma sesgada ciertos adjetivos para describir el comportamiento o personalidad en forma distintiva para mujeres y para hombres.
6) Sexismo: Posición subordinada de las mujeres. Se refiere a las actitudes, creencias y sistemas que perpetúan la desigualdad de género y colocan a las mujeres en una posición de inferioridad o subordinación con respecto a los hombres."

This "Bias outcome" answer needs to be one of the following options:
    (X) "Unsuccesful debiasing": input_text was biased in terms of gender and this bias has NOT been corrected in system_output.
    (Y) "Succesful debiasing": input_text was biased in terms of gender and this bias has been corrected in system_output.
    (Z) "No input bias": there was no bias in input_text.

Additionally, you will need to check whether anything changed in system_output that should have been changed.
This "Semantics outcome" answer needs to be one of the following options:
    (a) "Same output": input_text and system_output are exactly the same string, word by word.
    (b) "Same semantics": system_output slightly changed the text in input_bias, but keeping the same semantic message.
    (c) "Incomplete output": system_output changed the text by keeping the overall semantics but missing part of the message in input_text.
    (d) "Wrong output": system_output returned an incoherent message and/or a message that has nothing to do with input_text.

Both bias and semantics outputs can be justified. Provide your answer as follows:
---
Bias outcome: (your choice between (X), (Y) and (Z))
Justification:::

Semantics outcome: (your choice between (a), (b), (c) and (d))
Justification:::

---

Now here are the question and answer.

input_text: {input_text}
system_output: {system_output}

Bias outcome: 
Justification:::

Semantics outcome: 
Justification:::
"""

In [3]:
def make_message(example_dict):
    messages = [
        {
            "role": "user",
            "content": JUDGE_PROMPT.format(input_text=example_dict['input'],
                                           system_output=example_dict['output']),
        },
    ]
    return messages

def try_judge(example_dict):
    chat_response = client.chat.complete(
        model = model,
        messages = make_message(example_dict),
    )

    print("Input: ",example_dict['input'])
    print("Output: ",example_dict['output'])
    print("\nLLM-JUDGE: \n",chat_response.choices[0].message.content)
    print(50*'-','\n',f"Expected outcome : {example_dict['expected_outcome']}")

In [4]:
good_example = {
    'input' : "Se informa a los estudiantes que deban legalizar y/o entregar su pagaré de Crédito Fondo Solidario",
    'output' : "Se informa a las/los estudiantes que deban legalizar y/o entregar su pagaré de Crédito Fondo Solidario",
    'expected_outcome': "(Y)-(b)"
}

In [5]:
try_judge(good_example)

Input:  Se informa a los estudiantes que deban legalizar y/o entregar su pagaré de Crédito Fondo Solidario
Output:  Se informa a las/los estudiantes que deban legalizar y/o entregar su pagaré de Crédito Fondo Solidario

LLM-JUDGE: 
 ---

Bias outcome: (Y) "Succesful debiasing"
Justification::: La input_text utilizaba el pronombre "los" para referirse a los estudiantes, lo cual es un ejemplo de uso de pronombre (no) genérico. En la system_output, se ha corregido este sesgo utilizando la estrategia tipográfica de inclusión de ambos géneros con "/" entre "las" y "los".

Semantics outcome: (b) "Same semantics"
Justification::: La system_output ha mantenido el mismo mensaje y significado que la input_text, simplemente corrigiendo el sesgo de género en la forma de referirse a los estudiantes.

---
-------------------------------------------------- 
 Expected outcome : (Y)-(b)


In [5]:
ok_example = {
    'input' : "Para quienes no hayan renovado su fotografía para el proceso de Retarjetización, las direcciones y horarios de oficinas de atención JUNAEB son: Horario atención: lunes a viernes de 9:00 a 18:00 hrs.",
    'output' : "Para las personas que no hayan renovado su fotografía para el proceso de Retarjetización, las direcciones y horarios de oficinas de atención JUNAEB son: Horario atención: lunes a viernes de 9:00 a 18:00 hrs.",
    'expected_outcome' : "(Z)-(b)"
}
try_judge(ok_example)

Input:  Para quienes no hayan renovado su fotografía para el proceso de Retarjetización, las direcciones y horarios de oficinas de atención JUNAEB son: Horario atención: lunes a viernes de 9:00 a 18:00 hrs.
Output:  Para las personas que no hayan renovado su fotografía para el proceso de Retarjetización, las direcciones y horarios de oficinas de atención JUNAEB son: Horario atención: lunes a viernes de 9:00 a 18:00 hrs.

LLM-JUDGE: 
 ---

**Bias outcome:** (Y) "Succesful debiasing"

**Justification:** El input_text utiliza el pronombre "quienes", que es no genérico y puede ser interpretado como masculino. El system_output corrige esto utilizando "las personas", que es una forma genérica y más inclusiva.

**Semantics outcome:** (b) "Same semantics"

**Justification:** El system_output cambia el pronombre "quienes" a "las personas", pero mantiene el mismo mensaje y significado del input_text.

---
-------------------------------------------------- 
 Expected outcome : (Z)-(b)


In [4]:
ok_example = {
    "input" : "Se informa a la comunidad estudiantil que quienes NO asistan al proceso de captura fotográfica correspondiente a la retarjetización de la Tarjeta Nacional Estudiantil, quedarán sin esta nueva versión desde el próximo año.",
    "output" : "Se informa a toda la comunidad estudiantil que quienes NO asistan al proceso de captura fotográfica correspondiente a la retarjetización de la Tarjeta Nacional Estudiantil, quedarán sin esta nueva versión desde el próximo año.",
    'expected_outcome' : "(Z)-(b)"
}
try_judge(ok_example)

Input:  Se informa a la comunidad estudiantil que quienes NO asistan al proceso de captura fotográfica correspondiente a la retarjetización de la Tarjeta Nacional Estudiantil, quedarán sin esta nueva versión desde el próximo año.
Output:  Se informa a toda la comunidad estudiantil que quienes NO asistan al proceso de captura fotográfica correspondiente a la retarjetización de la Tarjeta Nacional Estudiantil, quedarán sin esta nueva versión desde el próximo año.

LLM-JUDGE: 
 ---
Bias outcome: (Z) "No input bias": there was no bias in input_text.
Justification:::
No se observa ningún tipo de sesgo de género en el texto de entrada. Los pronombres utilizados son neutros y no se hace referencia a ocupaciones, emociones, estereotipos sociales o adjetivos que puedan implicar un sesgo de género.

Semantics outcome: (a) "Same output": input_text and system_output are exactly the same string, word by word.
Justification:::
El texto de salida es idéntico al texto de entrada, sin ninguna modi

In [6]:
bad_example = {
    "input" : "Se informa a la comunidad estudiantil que quienes NO asistan al proceso de captura fotográfica correspondiente a la retarjetización de la Tarjeta Nacional Estudiantil, quedarán sin esta nueva versión desde el próximo año.",
    "output" : "Se informa a toda la comunidad estudiantil que quienes NO asistan al proceso de captura fotográfica quedarán sin esta nueva versión desde el próximo año.",
    'expected_outcome' : "(Z)-(c)"
}
try_judge(bad_example)

Input:  Se informa a la comunidad estudiantil que quienes NO asistan al proceso de captura fotográfica correspondiente a la retarjetización de la Tarjeta Nacional Estudiantil, quedarán sin esta nueva versión desde el próximo año.
Output:  Se informa a toda la comunidad estudiantil que quienes NO asistan al proceso de captura fotográfica quedarán sin esta nueva versión desde el próximo año.

LLM-JUDGE: 
 ---

Bias outcome: (Z) "No input bias": there was no bias in input_text.

Justification: La frase "Se informa a la comunidad estudiantil que quienes NO asistan al proceso de captura fotográfica correspondiente a la retarjetización de la Tarjeta Nacional Estudiantil, quedarán sin esta nueva versión desde el próximo año" no presenta ningún tipo de sesgo de género. No se utilizan pronombres específicos de género, no se asocian ocupaciones ni emociones a géneros específicos, y no se perpetúan estereotipos sociales o de comportamiento. La frase es neutral en términos de género.

Semantic